# Preamble

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import InterpolationMode
import torchvision.utils as vutils
from torchview import draw_graph


## Constants

In [ ]:
DEBUG = True
RESET = False

IMG_H = IMG_W = 256
RANDOM_SEED = 23                # mj
K = 5
STYLE_TOGGLE = True
TUNING_TOGGLE = False


## Style

In [ ]:
if STYLE_TOGGLE:                        # Dark used to visualize in Emacs
    sns.set_theme(style="darkgrid")

    plt.style.use("dark_background")
    plt.rcParams.update({
        "figure.facecolor": "#282C34",  # whole figure background
        "axes.facecolor": "#282C34",    # axes background
        "axes.edgecolor": "white",
        "axes.labelcolor": "white",
        "xtick.color": "white",
        "ytick.color": "white",
        "text.color": "white",
        "axes.titlecolor": "white",
        "grid.color": "#444444",
        "lines.linewidth": 2,
    })
else:                                   # Light used for report
    plt.style.use("default")
    plt.rcParams.update({
        "figure.facecolor": "white",    # whole figure background
        "axes.facecolor": "white",      # axes background
        "axes.edgecolor": "black",
        "axes.labelcolor": "black",
        "xtick.color": "black",
        "ytick.color": "black",
        "text.color": "black",
        "axes.titlecolor": "black",
        "grid.color": "#262626",
        "lines.linewidth": 2,
    })


## Info & Reproducibility

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently using: {device}")

torch.manual_seed(RANDOM_SEED)                                          # Reproducibility

dataset = datasets.ImageFolder(root="archive", transform=transforms.ToTensor())
loader = DataLoader(dataset, batch_size=1, shuffle=False)

mean = diff = 0.0
num_pixels = 0

for image, _ in loader:
    image = image.squeeze(0)                                            # Remove useless batch=1
    mean += image.sum(dim=[1, 2])
    num_pixels += image.shape[1] * image.shape[2]                       # Height * Width

mean /= num_pixels

for image, _ in loader:
    image = image.squeeze(0)                                            # Remove useless batch=1
    diff += ((image - mean.view(-1, 1, 1)) ** 2).sum(dim=[1, 2])

std = (diff / num_pixels).sqrt()

print("Dataset mean:", mean)
print("Dataset std:", std)


# Preprocessing and Architectures

## Transform

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_H, IMG_W)
                    , interpolation=InterpolationMode.BICUBIC),         # Image Resizing using bicubic interpolation
    transforms.RandomHorizontalFlip(),                                  # Randomly flip Horizontally (0.5)
    transforms.RandomAffine(
        degrees=90,                                                     # -90 and +90
        scale=(0.85, 1.15),                                             # zoom 85% or zoom 115%
    ),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),                                              # Scaling [0,255] -> [0,1]
    transforms.Normalize(mean=mean,                                     # output[channel] = (input[channel] - mean[channel]) / std[channel]
                        std=std)
])


## Architectures

In [ ]:
class CNN_First(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3,   32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32,  64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.ln1 = nn.Linear(128*32*32, 256)                            # After flattening
        self.ln2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)

        x = F.relu(self.ln1(x))
        x = self.ln2(x)
        return x

class CNN_Second(CNN_First):
    def __init__(self):
        super().__init__()

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.dropout2d = nn.Dropout2d(p=0.1)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout2d(x)

        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2d(x)

        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout2d(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.ln1(x))
        x = self.dropout(x)
        x = self.ln2(x)
        return x

class CNN_Third(CNN_Second):
    def __init__(self):
        super().__init__()

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.bn4 = nn.BatchNorm2d(256)

        self.ln1 = nn.Linear(256*16*16, 1024)
        self.ln2 = nn.Linear(1024, 256)
        self.ln3 = nn.Linear(256, 3)

    def forward(self, x):

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.ln1(x))
        x = self.dropout(x)
        x = F.relu(self.ln2(x))
        x = self.dropout(x)
        x = self.ln3(x)
        return x

class CNN_Fourth(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(64)
        self.bn5 = nn.BatchNorm2d(128)
        self.bn6 = nn.BatchNorm2d(128)
        self.bn7 = nn.BatchNorm2d(256)
        self.bn8 = nn.BatchNorm2d(256)
        self.bn9 = nn.BatchNorm2d(512)
        self.bn10 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout2d = nn.Dropout2d(0.1)
        self.dropout = nn.Dropout(0.25)

        self.ln1 = nn.Linear(512*8*8, 1024)
        self.ln2 = nn.Linear(1024, 256)
        self.ln3 = nn.Linear(256, 3)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        # Block 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        # Block 3
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        # Block 4
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        # Block 5
        x = F.relu(self.bn9(self.conv9(x)))
        x = F.relu(self.bn10(self.conv10(x)))
        x = self.pool(x)
        x = self.dropout2d(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected
        x = F.relu(self.ln1(x))
        x = self.dropout(x)
        x = F.relu(self.ln2(x))
        x = self.dropout(x)
        x = self.ln3(x)
        return x


# Training

In [ ]:
dataset = datasets.ImageFolder(root="archive", transform=transform)

train_idx, test_idx = train_test_split(
    list(range(len(dataset))),
    test_size=0.15,
    random_state=RANDOM_SEED,
    stratify=dataset.targets
)

train_subset = Subset(dataset, train_idx)
test_subset = Subset(dataset, test_idx)

kf = KFold(n_splits=K, shuffle=True, random_state=RANDOM_SEED)

architecture = CNN_Fourth                                        # Change this to change architecture

hyperparams = {
    "lr": [1e-3, 5e-4, 1e-4],
    "batch_size": [16, 32, 64],
    "epochs": [10, 20, 30]
}

param_combinations = list(itertools.product(
    hyperparams["lr"],
    hyperparams["batch_size"],
    hyperparams["epochs"]
))


if TUNING_TOGGLE:
    best_loss = float("inf")
    best_params = None

    for lr, batch_size, epochs in param_combinations:
        print(f"Testing: lr={lr}, batch_size={batch_size}, epochs={epochs}")
        fold_losses = []
        fold_f1s = []
        fold_accuracies = []
        fold_recalls = []
        fold_precisions = []

        train_losses_per_fold = []
        val_losses_per_fold = []

        for fold, (t_idx, v_idx) in enumerate(kf.split(train_subset)):

            train_fold_subset = Subset(train_subset, t_idx)
            val_fold_subset = Subset(train_subset, v_idx)

            train_loader = DataLoader(train_fold_subset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_fold_subset, batch_size=batch_size)

            model = architecture().to(device)
            criterion = nn.CrossEntropyLoss()                           # default reduction='mean'
            optimizer = optim.Adam(model.parameters(), lr=lr)

            val_loss = 0.0
            all_preds = []
            all_labels = []

            train_losses = []
            val_losses = []

            for epoch in range(epochs):
                model.train()
                train_loss = 0.0

                for images, labels in train_loader:
                    images, labels = images.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() * images.size(0)

                train_loss /= len(train_fold_subset)
                train_losses.append(train_loss)

                model.eval()
                val_loss = 0.0
                all_preds = []
                all_labels = []

                with torch.no_grad():
                    for images, labels in val_loader:
                        images, labels = images.to(device), labels.to(device)
                        outputs = model(images)
                        loss = criterion(outputs, labels)                       # mean loss per batch
                        val_loss += loss.item() * images.size(0)                # total loss for batch
                        if (epoch == epochs - 1):
                            preds = torch.argmax(outputs, dim=1)
                            all_preds.extend(preds.cpu().numpy())
                            all_labels.extend(labels.cpu().numpy())

                val_loss /= len(val_fold_subset)
                val_losses.append(val_loss)

            fold_losses.append(val_loss)

            train_losses_per_fold.append(train_losses)
            val_losses_per_fold.append(val_losses)

            cm = confusion_matrix(all_labels, all_preds, labels=list(range(3)))
            TP = cm.diagonal()
            FP = cm.sum(axis=0) - TP
            FN = cm.sum(axis=1) - TP
            TN = cm.sum() - (TP + FP + FN)

            precision_per_class = TP / (TP + FP + 1e-8)
            recall_per_class = TP / (TP + FN + 1e-8)
            f1_per_class = 2 * precision_per_class * recall_per_class / (precision_per_class + recall_per_class + 1e-8)

            precision_macro = precision_per_class.mean()
            recall_macro = recall_per_class.mean()
            f1_macro = f1_per_class.mean()
            accuracy = TP.sum() / cm.sum()

            fold_precisions.append(precision_macro)
            fold_recalls.append(recall_macro)
            fold_f1s.append(f1_macro)
            fold_accuracies.append(accuracy)

            print(f"Fold {fold+1} | Loss: {val_loss:.4f}, "
            f"Precision: {precision_macro:.4f}, "
            f"Recall: {recall_macro:.4f}, "
            f"F1: {f1_macro:.4f}, "
            f"Accuracy: {accuracy:.4f}")

        avg_cv_loss = sum(fold_losses) / K
        avg_precisions = sum(fold_precisions) / K
        avg_f1 = sum(fold_f1s) / K
        avg_accuracy = sum(fold_accuracies) / K
        avg_recall = sum(fold_recalls) / K

        print(f"Average CV | Loss: {avg_cv_loss:.4f}, "
        f"Precision: {avg_precisions:.4f}, "
        f"F1: {avg_f1:.4f}, "
        f"Accuracy: {avg_accuracy:.4f}, "
        f"Recall: {avg_recall:.4f}\n")

        train_losses_per_fold = np.array(train_losses_per_fold)  # shape: (5, epochs)
        val_losses_per_fold = np.array(val_losses_per_fold)

        epochs_range = range(1, epochs+1)
        num_folds = train_losses_per_fold.shape[0]

        fold_colors = ['blue', 'green', 'orange', 'purple', 'brown']

        plt.figure(figsize=(16,9))

        for fold in range(num_folds):
            color = fold_colors[fold % len(fold_colors)]
            plt.plot(epochs_range, train_losses_per_fold[fold], color=color, linestyle='dashdot', label=f'Train Fold {fold+1}')
            plt.plot(epochs_range, val_losses_per_fold[fold], color=color, linestyle='solid', label=f'Val Fold {fold+1}')

        plt.plot(epochs_range, train_losses_per_fold.mean(axis=0), color='black', linestyle='dashdot', linewidth=2, label='Train Mean')
        plt.plot(epochs_range, val_losses_per_fold.mean(axis=0), color='black', linestyle='solid', linewidth=2, label='Val Mean')

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'lr={lr}, batch_size={batch_size}, epochs={epochs}')
        plt.legend()
        plt.show()

        if avg_cv_loss < best_loss:
            best_loss = avg_cv_loss
            best_params = (lr, batch_size, epochs)

    print("Best hyperparameters:")
    print(f"Learning rate: {best_params[0]}")
    print(f"Batch size: {best_params[1]}")
    print(f"Epochs: {best_params[2]}")
    print(f"Average CV loss: {best_loss:.4f}")
else:
    best_params = [1e-4, 16, 30]

final_model = architecture().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params[0])
final_epochs = best_params[2]

train_loader = DataLoader(train_subset, batch_size=best_params[1], shuffle=True)
test_loader = DataLoader(test_subset, batch_size=best_params[1])

train_losses_final = []
test_losses_final = []
train_accuracies_final = []
test_accuracies_final = []

for epoch in range(final_epochs):
    final_model.train()
    epoch_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() * images.size(0)

        preds = torch.argmax(outputs, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_train_loss /= len(train_subset)
    train_losses_final.append(epoch_train_loss)
    train_accuracies_final.append(correct_train / total_train)

    final_model.eval()
    epoch_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            loss = criterion(outputs, labels)
            epoch_test_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, dim=1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)

    epoch_test_loss /= len(test_subset)
    test_losses_final.append(epoch_test_loss)
    test_accuracies_final.append(correct_test / total_test)

final_model.eval()
all_preds = []
all_labels = []
test_loss = 0.0
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_subset)

cm = confusion_matrix(all_labels, all_preds, labels=list(range(3)))
TP = cm.diagonal()
FP = cm.sum(axis=0) - TP
FN = cm.sum(axis=1) - TP
TN = cm.sum() - (TP + FP + FN)

precision_per_class = TP / (TP + FP + 1e-8)
recall_per_class = TP / (TP + FN + 1e-8)
f1_per_class = 2 * precision_per_class * recall_per_class / (precision_per_class + recall_per_class + 1e-8)

precision_macro = precision_per_class.mean()
recall_macro = recall_per_class.mean()
f1_macro = f1_per_class.mean()
accuracy = TP.sum() / cm.sum()

epochs_range = range(1, final_epochs+1)
plt.figure(figsize=(16,9))
plt.plot(epochs_range, train_losses_final, linestyle='dashdot', color='blue', label='Train Loss')
plt.plot(epochs_range, test_losses_final, linestyle='solid', color='red', label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curves (Loss)')
plt.legend()
plt.show()

plt.figure(figsize=(16,9))
plt.plot(epochs_range, train_accuracies_final, linestyle='dashdot', color='blue', label='Train Acc')
plt.plot(epochs_range, test_accuracies_final, linestyle='solid', color='red', label='Test Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Learning Curves (Loss)')
plt.legend()
plt.show()

print("Final Test Set Performance:")
print(f"Loss:      {test_loss:.4f}")
print(f"Precision: {precision_macro:.4f}")
print(f"Recall:    {recall_macro:.4f}")
print(f"F1:        {f1_macro:.4f}")
print(f"Accuracy:  {accuracy:.4f}")


## Generalization

In [ ]:
data_dir = "Generalize_Hands"

transformGen = transforms.Compose([
    transforms.Resize((IMG_H, IMG_W)
                      , interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

dataset = datasets.ImageFolder(root=data_dir, transform=transformGen)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

final_model.eval()
final_model.to(device)

all_results = []

for inputs, labels in dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = final_model(inputs)
        pred = torch.argmax(outputs, dim=1)
    all_results.append((inputs.cpu()[0], labels.item(), pred.item(), outputs.cpu()[0]))

plt.figure(figsize=(40, 10))
for i, (img, true_label, pred_label, logits) in enumerate(all_results):
    plt.subplot(1, len(all_results), i+1)
    inv_normalize = transforms.Normalize(
        mean=[-0.3229/0.2554, -0.5491/0.1014, -0.2601/0.1327],
        std=[1/0.2554, 1/0.1014, 1/0.1327]
    )
    img_show = inv_normalize(img)
    img_show = transforms.ToPILImage()(img_show)
    plt.imshow(img_show)

    color = "green" if pred_label == true_label else "red"

    logits_text = "\n".join([f"{c}: {l:.2f}" for c, l in zip(dataset.classes, logits)])
    plt.title(f"T: {dataset.classes[true_label]}\nP: {dataset.classes[pred_label]}\n{logits_text}", fontsize=8, c=color)
    plt.axis("off")
plt.show()

true_labels = [t for _, t, _, _ in all_results]
pred_labels = [p for _, _, p, _ in all_results]

cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset.classes)

plt.figure(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45, values_format='d')
plt.title("Confusion Matrix")
plt.show()


## Plot Architectures

In [ ]:
model_graph = draw_graph(CNN_Fourth(), input_size=(1,3,256,256))
model_graph.visual_graph


# Reset

In [ ]:
if RESET:
    %reset
